In [4]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
from dotenv import load_dotenv

#Reading the content of .env
load_dotenv('.env')

True

In [ ]:
api_key = os.environ['GOOGLE_MAPS_API_KEY']


In [7]:
import requests

In [8]:
import urllib

In [9]:
from models import toilet

In [10]:
#Turn address into latitude and longitude
def get_lat_lng(address):
    api_key = os.environ['GOOGLE_MAPS_API_KEY']
    url = 'https://maps.googleapis.com/maps/api/geocode/json?address={}&key={}'.format(address, api_key)
    response = requests.get(url)
    response_json = response.json()
    lat = response_json["results"][0]["geometry"]["location"]["lat"]
    lng = response_json["results"][0]["geometry"]["location"]["lng"]
    return lat, lng



In [11]:
state = "NSW"
#Call Public Toilet API
def get_toilet_data(lat, lng, state):
    #url = 'https://data.gov.au/data/api/3/action/datastore_search?resource_id=34076296-6692-4e30-b627-67b7c4eb1027&q=state:{state}&limit=30000"
    #url = f'https://data.gov.au/data/api/3/action/datastore_search?resource_id=34076296-6692-4e30-b627-67b7c4eb1027&limit=30000&q=state:{state}'
    url = "https://data.gov.au/data/api/3/action/datastore_search_sql?sql=SELECT%20*%20from%20%2234076296-6692-4e30-b627-67b7c4eb1027%22%20WHERE%20state%20LIKE%20%27{}%27".format(state)
    #url = f"https://data.gov.au/data/api/3/action/datastore_search?resource_id=34076296-6692-4e30-b627-67b7c4eb1027&full_text=state:{state}&limit=30000"
    response = requests.get(url)
    response_json = response.json()
    return response_json



In [82]:
state = "NSW"
#Call Public Toilet API
#このURLを作りたい →　https://data.gov.au/data/api/3/action/datastore_search?resource_id=34076296-6692-4e30-b627-67b7c4eb1027&limit=30000&filters={%22State%22:%20%22NSW%22}
url = "https://data.gov.au/data/api/3/action/datastore_search?resource_id=34076296-6692-4e30-b627-67b7c4eb1027&filters={}&limit=100".format("{\"State\":"+"\""+state+"\""+"}")
#url = f'https://data.gov.au/data/api/3/action/datastore_search?resource_id=34076296-6692-4e30-b627-67b7c4eb1027&limit=30000&filters=State:{}'.format()
#url = "https://data.gov.au/data/api/3/action/datastore_search_sql?sql=SELECT%20*%20from%20%2234076296-6692-4e30-b627-67b7c4eb1027%22%20WHERE%20%27State%27%20LIKE%20%27{}%27".format(state)
#url = f"https://data.gov.au/data/api/3/action/datastore_search?resource_id=34076296-6692-4e30-b627-67b7c4eb1027&full_text=state:{state}&limit=30000"
response = requests.get(url)
response_json = response.json()
print(url)
print(response_json)


https://data.gov.au/data/api/3/action/datastore_search?resource_id=34076296-6692-4e30-b627-67b7c4eb1027&filters={"State":"NSW"}&limit=100
{'help': 'https://data.gov.au/data/api/3/action/help_show?name=datastore_search', 'success': True, 'result': {'filters': {'State': 'NSW'}, 'include_total': True, 'limit': 100, 'records_format': 'objects', 'resource_id': '34076296-6692-4e30-b627-67b7c4eb1027', 'total_estimation_threshold': None, 'records': [{'_id': 205, 'FacilityID': '272', 'URL': 'https://toiletmap.gov.au/facility/272', 'Name': 'Brisbane Street', 'FacilityType': 'Park or reserve', 'Address1': 'Brisbane Street', 'Town': 'Merriwa', 'State': 'NSW', 'AddressNote': '', 'Latitude': '-32.13972275', 'Longitude': '150.36090207', 'Parking': 'False', 'ParkingAccessible': 'False', 'ParkingNote': '', 'KeyRequired': 'False', 'MLAK24': 'False', 'MLAKAfterHours': 'False', 'PaymentRequired': 'False', 'AccessNote': 'access over grass', 'AdultChange': 'False', 'ChangingPlaces': 'False', 'BYOSling': 'Fa

In [83]:
def sort_by_distance(toilet_data, lat, lng):
    # 1. toilet_dataをリスト型に変換する(toilet_listにする)
    toilet_list = list(toilet_data.items())

    # 2. toilet_listの配列3つ目の要素にトイレ情報が全て入っているので、これを分解してトイレごとのリストを作成する
    #toilet_info_list = toilet_list[2][1]['records']

    # 3. トイレごとのリストの各要素について、緯度と経度をstring型からfloat型に変換する
    for toilet_info in toilet_info_list:
        toilet_info['Latitude'] = float(toilet_info['Latitude'])
        toilet_info['Longitude'] = float(toilet_info['Longitude'])

    # 4. また、各要素の緯度経度と、インプットした緯度経度からユークリッド距離を計算し、distanceの要素として値を追加する
    import math
    from geopy.distance import geodesic
    # Calculating distance in meter by Hybeny's Distance Formula 
    def calculate_distance(lat, lng, toilet_info_lat, toilet_info_lng):
        pole_radius = 6356752.314245
        equator_radius = 6378137.0
        # Convert to radians
        lat = math.radians(lat)
        lng = math.radians(lng)
        toilet_info_lat = math.radians(toilet_info_lat)
        toilet_info_lng = math.radians(toilet_info_lng)
        lat_difference = lat - toilet_info_lat
        lon_difference = lng - toilet_info_lng
        lat_average = (lat + toilet_info_lat) / 2
        e2 = (math.pow(equator_radius, 2) - math.pow(pole_radius, 2)) / math.pow(equator_radius, 2)
        w = math.sqrt(1- e2 * math.pow(math.sin(lat_average), 2))
        m = equator_radius * (1 - e2) / math.pow(w, 3)
        n = equator_radius / w
        distance = math.sqrt(math.pow(m * lat_difference, 2) + math.pow(n * lon_difference * math.cos(lat_average), 2))
        return distance
    for toilet_info in toilet_info_list:
        #toilet_info['distance'] = calculate_distance(lat, user_lon, toilet_info['Latitude'], toilet_info['Longitude'])
        toilet_info['distance'] = calculate_distance(lat, lng, toilet_info['Latitude'], toilet_info['Longitude'])
        
    # 5. distanceの値が小さい順にリストをソートする
    toilet_info_list_sorted = sorted(toilet_info_list, key=lambda x: x['distance'])
    return toilet_info_list_sorted
    

In [84]:
#Return the top 15 items of the sorted list
import re
address = f"2+Dind+St%2C+Milsons+Point+NSW+2061"
match = re.search(r"\bNSW\b|\bVIC\b|\bQLD\b|\bWA\b|\bSA\b|\bTAS\b|\bACT\b|\bNT\b", address)
state = match.group()
print(state)

lat, lng = get_lat_lng(address)
toilet_data = get_toilet_data(lat, lng, state)
sorted_toilet_data = sort_by_distance(toilet_data, lat, lng)
print(sorted_toilet_data[:15])

NSW
{'help': 'https://data.gov.au/data/api/3/action/help_show?name=datastore_search_sql', 'error': {'query': ['(psycopg2.errors.UndefinedColumn) column "state" does not exist\nLINE 1: ...from "34076296-6692-4e30-b627-67b7c4eb1027" WHERE state LIKE...\n                                                             ^\nHINT:  Perhaps you meant to reference the column "34076296-6692-4e30-b627-67b7c4eb1027.State".\n\n[SQL: SELECT * FROM (SELECT * from "34076296-6692-4e30-b627-67b7c4eb1027" WHERE state LIKE \'NSW\') AS blah LIMIT 32001 ;]\n(Background on this error at: http://sqlalche.me/e/f405)'], 'info': {'statement': ['SELECT * FROM (SELECT * from "34076296-6692-4e30-b627-67b7c4eb1027" WHERE state LIKE \'NSW\') AS blah LIMIT 32001 ;'], 'params': [{}], 'orig': ['column "state" does not exist\nLINE 1: ...from "34076296-6692-4e30-b627-67b7c4eb1027" WHERE state LIKE...\n                                                             ^\nHINT:  Perhaps you meant to reference the column "34076296-669

AttributeError: 'NoneType' object has no attribute 'items'

In [ ]:
print(state)

NSW
